# Our experiments

### from paper

In [1]:
import os
import torch
from src.experiments.run_experiments import setup_environment, CONFIG, run_realworld_experiment, run_synthetic_trial, plot_results

setup_environment(CONFIG)
results = {"synthetic": {}, "real_world": {}}
    
# print("Starting synthetic experiments")
# for func in CONFIG["synthetic"]["functions"]:
#     func_results = []
#     for trial in range(CONFIG["synthetic"]["trials"]):
#         try:
#             print(f"Running {func.__name__} trial {trial+1}/{CONFIG['synthetic']['trials']}")
#             result = run_synthetic_trial(func, CONFIG)
#             func_results.append(result)
#         except Exception as e:
#             print(f"Trial failed for {func.__name__}: {str(e)}")
#             continue

#         results["synthetic"][func.__name__] = func_results
    
print("\nStarting real-world experiments")
for dataset in CONFIG["real_world"]["datasets"]:
    results["real_world"][dataset] = run_realworld_experiment(dataset, CONFIG)

torch.save(results, os.path.join(CONFIG["paths"]["results"], "full_results.pt"))
plot_results(results, CONFIG)


Starting real-world experiments
Processing cal_housing
Succesfully Loaded cal_housing!
[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]] [4.526 3.585 3.521 ... 0.923 0.847 0.894] data
{'train': array([[ 2.34476576,  0.98214266,  0.62855945, ..., -0.04959654,
         1.05254828, -1.32783522],
       [ 2.33223796, -0.60701891,  0.32704136, ..., -0.09251223,
         1.04318455, -1.32284391],
       [ 1.7826994 ,  1.85618152,  1.15562047, ..., -0.02584253,
         1.03850269, -1.332

KeyboardInterrupt: 

# Experiment from github

In [1]:
import torch
import numpy as np
from neural_interaction_detection import get_interactions
from multilayer_perceptron import MLP, train, get_weights
from utils import preprocess_data, get_pairwise_auc, get_anyorder_R_precision, set_seed, print_rankings

## Generate synthetic data with ground truth interactions

In [2]:
use_main_effect_nets = True # toggle this to use "main effect" nets
num_samples = 30000
num_features = 10

In [3]:
def synth_func(X):
    X1, X2, X3, X4, X5, X6, X7, X8, X9, X10 = X.transpose()

    interaction1 = np.exp(np.abs(X1-X2))                        
    interaction2 = np.abs(X2*X3)  
    interaction3 = -1*(X3**2)**np.abs(X4) 
    interaction4 = (X1*X4)**2
    interaction5 = np.log(X4**2 + X5**2 + X7**2 + X8**2)
    main_effects = X9 + 1/(1 + X10**2)

    Y =              interaction1 + interaction2 + interaction3 + interaction4 + interaction5 + main_effects
    ground_truth = [     {1,2},        {2,3},         {3,4},         {1,4},        {4,5,7,8}     ]
    
    return Y, ground_truth

In [4]:
set_seed(42)
X = np.random.uniform(low=-1, high=1, size=(num_samples,num_features))
Y, ground_truth = synth_func(X)
data_loaders = preprocess_data(X, Y, valid_size=10000, test_size=10000, std_scale=True, get_torch_loaders=True)

## Train a multilayer perceptron (MLP)

In [5]:
device = torch.device("cpu")
model = MLP(num_features, [140, 100, 60, 20], use_main_effect_nets=use_main_effect_nets).to(device)

In [6]:
model, mlp_loss = train(model, data_loaders, device=device, learning_rate=1e-2, l1_const = 5e-5, verbose=True)

starting to train
early stopping enabled
[epoch 1, total 100] train loss: 0.1921, val loss: 0.0548
[epoch 3, total 100] train loss: 0.0290, val loss: 0.0283
[epoch 5, total 100] train loss: 0.0239, val loss: 0.0557
[epoch 7, total 100] train loss: 0.0151, val loss: 0.0168
[epoch 9, total 100] train loss: 0.0143, val loss: 0.0184
[epoch 11, total 100] train loss: 0.0116, val loss: 0.0083
[epoch 13, total 100] train loss: 0.0123, val loss: 0.0117
[epoch 15, total 100] train loss: 0.0104, val loss: 0.0094
[epoch 17, total 100] train loss: 0.0077, val loss: 0.0137
[epoch 19, total 100] train loss: 0.0083, val loss: 0.0139
[epoch 21, total 100] train loss: 0.0070, val loss: 0.0054
[epoch 23, total 100] train loss: 0.0091, val loss: 0.0063
[epoch 25, total 100] train loss: 0.0111, val loss: 0.0099
[epoch 27, total 100] train loss: 0.0064, val loss: 0.0068
early stopping!
Finished Training. Test loss:  0.005764756351709366


## Get the MLP's learned weights

In [7]:
model_weights = get_weights(model)

## Detect interactions from the weights

In [8]:
anyorder_interactions = get_interactions(model_weights, one_indexed=True)
pairwise_interactions = get_interactions(model_weights, pairwise=True, one_indexed=True)

        
print_rankings(pairwise_interactions, anyorder_interactions, top_k=10, spacing=14)

Pairwise interactions              Arbitrary-order interactions
(1, 2)        7.8430                      (1, 2)        6.8951        
(4, 8)        3.1959                      (2, 3)        2.0953        
(5, 8)        3.0521                      (7, 8)        1.7971        
(7, 8)        3.0290                      (4, 5, 8)     1.6026        
(4, 5)        2.8506                      (1, 4)        1.5912        
(2, 3)        2.6294                      (5, 7)        1.5261        
(1, 4)        2.5037                      (3, 4)        1.3500        
(5, 7)        2.4460                      (4, 7)        1.0580        
(4, 7)        2.2369                      (4, 7, 8)     0.7727        
(3, 4)        1.8870                      (4, 5, 7, 8)  0.5467        


## Evaluate the interactions

In [9]:
auc = get_pairwise_auc(pairwise_interactions, ground_truth)
r_prec = get_anyorder_R_precision(anyorder_interactions, ground_truth)

print("Pairwise AUC", auc, ", Any-order R-Precision", r_prec)

Pairwise AUC 1.0 , Any-order R-Precision 1.0
